# Pipelines Test Library Notebook

# Imports

In [1]:
# Import piplines

from binary_nets import binary_nets_wrapper # this is actually pipeline1
from pipeline2 import pipeline2
from pipeline3 import pipeline3
from pipeline4 import pipeline4

from ensemble_builder import ensemble_builder

import image_utils as iu

#Import datasets
import dataset_generator as dataset

from time import time
import os

Using TensorFlow backend.


# Data Load

In [2]:
(X_train_ocr, y_train_ocr, X_test_ocr, y_test_ocr, _) = dataset.generate_all_chars_with_class(verbose=0, plot=False)
print("Loaded dataset for all characters")

(X_train_cut, y_train_cut, X_test_cut, y_test_cut) = dataset.generate_dataset_for_segmentator(verbose=0, plot=False)
print("Loaded dataset for segmentator")

X_train_char = {}
y_train_char = {}
X_test_char = {}
y_test_char = {}

for char in dataset.ALPHABET_ALL:
    (X_train_char[char], y_train_char[char], X_test_char[char], y_test_char[char]) = \
                                                    dataset.generate_positive_and_negative_labeled(char, verbose=0)
    print("Loaded dataset for character " + char)

Loaded dataset for all characters
Loaded dataset for segmentator
Loaded dataset for character a
Loaded dataset for character c
Loaded dataset for character d_mediana
Loaded dataset for character e
Loaded dataset for character i
Loaded dataset for character m
Loaded dataset for character n
Loaded dataset for character o
Loaded dataset for character r
Loaded dataset for character s_mediana
Loaded dataset for character t
Loaded dataset for character u
Loaded dataset for character d_alta
Loaded dataset for character s_alta
Loaded dataset for character b
Loaded dataset for character f
Loaded dataset for character h
Loaded dataset for character l
Loaded dataset for character g
Loaded dataset for character p
Loaded dataset for character q
Loaded dataset for character s_bassa


## Setting and training of the nets

## Binary nets

In [3]:
'''
path_b_nets = "checkpoints/testpip/pipeline1"

# binary nets
binary_nets = {}

for letter in dataset.ALPHABET_ALL:
    # Create the binary net
    letter_path = os.path.join(path_b_nets, letter)
    binary_nets[letter]= ensemble_builder(2, 2, number_of_nets=2, \
                path=letter_path, nb_filters1=20, nb_filters2=40, dense_layer_size1=150)
    
    # Training
    binary_nets[letter].fit(X_train_char[letter], y_train_char[letter], X_test_char[letter], y_test_char[letter], forceRetrain=False)
'''

'\npath_b_nets = "checkpoints/testpip/pipeline1"\n\n# binary nets\nbinary_nets = {}\n\nfor letter in dataset.ALPHABET_ALL:\n    # Create the binary net\n    letter_path = os.path.join(path_b_nets, letter)\n    binary_nets[letter]= ensemble_builder(2, 2, number_of_nets=2,                 path=letter_path, nb_filters1=20, nb_filters2=40, dense_layer_size1=150)\n    \n    # Training\n    binary_nets[letter].fit(X_train_char[letter], y_train_char[letter], X_test_char[letter], y_test_char[letter], forceRetrain=False)\n'

## Cut classifier

In [4]:
path_cut_classifier = "checkpoints/letter_not_letter"

cut_class = ensemble_builder(2, 800, number_of_nets=2,\
                 path=path_cut_classifier, nb_filters1=50, nb_filters2=100,\
                 dense_layer_size1=250)

#cut_class.fit(X_train_cut, y_train_cut, X_test_cut, y_test_cut, forceRetrain=True)

## Ocr classifier

In [5]:
path_ocr_class = "checkpoints/09_22-classes"

ocr_classifier = ensemble_builder(22, 800, number_of_nets=2, path=path_ocr_class,\
            nb_filters1=50, nb_filters2=100, dense_layer_size1=250)

#ocr_classifier.fit(X_train_ocr, y_train_ocr, X_test_ocr, y_test_ocr, forceRetrain=False)

# Pipelines

## Pipeline 1

In [6]:
'''
pip1 = binary_nets_wrapper(binary_nets)

prediction_pip1 = pip1.predict(X_test_ocr)
'''

'\npip1 = binary_nets_wrapper(binary_nets)\n\nprediction_pip1 = pip1.predict(X_test_ocr)\n'


## Pipeline 2




In [7]:
pip2 = pipeline2(cut_class, ocr_classifier)

asseras_good_cuts = iu.open_many_samples( \
                    ["not_code/words/good_cuts/asseras/a1.png",
                     "not_code/words/good_cuts/asseras/f1.png",
                     "not_code/words/good_cuts/asseras/f2.png",
                     "not_code/words/good_cuts/asseras/e.png",
                     "not_code/words/good_cuts/asseras/r.png",
                     "not_code/words/good_cuts/asseras/a2.png",
                     "not_code/words/good_cuts/asseras/s.png"])

prediction_pip2 = pip2.predict(asseras_good_cuts)

In [8]:
prediction_pip2

[(True, array([('a',   9.99758005e+01), ('o',   1.13711780e-02),
         ('i',   7.22903933e-03)], 
        dtype=[('letters', '<U16'), ('grades', '<f8')])),
 (True,
  array([('s_alta',  96.79138064), ('l',   3.00127901), ('f',   0.20640711)], 
        dtype=[('letters', '<U16'), ('grades', '<f8')])),
 (False, []),
 (True, array([('e',  98.46054316), ('c',   1.17276963), ('o',   0.17953472)], 
        dtype=[('letters', '<U16'), ('grades', '<f8')])),
 (True, array([('r',  95.26131153), ('e',   3.89830396), ('i',   0.29342179)], 
        dtype=[('letters', '<U16'), ('grades', '<f8')])),
 (True, array([('a',   9.99343634e+01), ('o',   2.17619818e-02),
         ('i',   2.11800376e-02)], 
        dtype=[('letters', '<U16'), ('grades', '<f8')])),
 (True, array([('s_alta',   9.99579728e+01), ('f',   4.13175847e-02),
         ('l',   7.13807640e-04)], 
        dtype=[('letters', '<U16'), ('grades', '<f8')]))]

## Pipeline 3

In [9]:
'''
pip3 = pipeline3(pip1, ocr_classifier)

prediction_pip3 = pip3.predict(X_test_ocr)
'''

'\npip3 = pipeline3(pip1, ocr_classifier)\n\nprediction_pip3 = pip3.predict(X_test_ocr)\n'

## Pipeline 4

In [10]:
'''
pip4 = pipeline4(cut_class, pip1)

prediction_pip4 = pip4.predict(X_test_ocr)
'''

'\npip4 = pipeline4(cut_class, pip1)\n\nprediction_pip4 = pip4.predict(X_test_ocr)\n'